# Montecarlo Simulations - Valuating financial derivatives

## Summary

* Montecarlo Valuation for Binomial
* Montecarlo Valuation for Diffusion processes

Monte Carlo simulation is one of the most important algorithms with several applications in many fields, including quantitative finance as one of the most important. Specifically in finance, it can be used to price derivatives from all kinds such as options, futures, swaps, variance swaps, amongs other exotic ones. 

It consists of the calculation of repeated random samplings to make numerical estimations of unknown parameters, which in the case that matters to us is the log return of a given asset, for continuos time return, and consequently its price at the end of a simulation. The final objective is to apply the law of large numbers to obtain a value that is close to the final price of the derivative after M simulations of several trading periods. 

<img src='simulation_montecarlo_diffusion.png'>

In this task, computational power comes to hand as an important tool that allows us to perform thoursands of simulations of the M final payoffs of the derivative. In the following lines I'll process to demonstrate the application of this theory with a Python script.

## Import libraries

In [ ]:
from math import exp, log, sqrt
import datetime as dt
import numpy as np
from opciones import Analysis
import matplotlib.pyplot as plt 

Imported library "opciones" constains a black & scholes calculator which I'll  be using to compare values.

## Programming functions for general cases

In [13]:
def montecarlo_binominal(s0, strike, maturity, sigma, r, n, sims):
    """
    Calculates the price of an option call with montecarlo binomial case by introducing a set of parameters. 
    - s0: Underlying price at t0.
    - strike: Contract value of the call option.
    - maturity: period of time until the expiration of the contract in annual terms.
    - sigma: estimated constant volatility in annual terms.
    - r: estimated constant interest rate in annual terms.
    - sims: number of M simulations of the payoff of the call option.
    """
    delta = maturity / n
    m = r - sigma**2 / 2
    u = exp(m * delta + sigma * sqrt(delta))
    d = exp(m * delta - sigma * sqrt(delta))
    R = exp(r*delta)
    p = (R-d) / (u-d)   

    sims_vector = []
    payoff_final = []
    for w in range(sims):
        ran = np.random.rand(n)
        sims_vector.append(ran)

        log_sims = []
        for sim in ran:
            if sim < p:
                ret = log(d)
                log_sims.append(ret)
            elif sim > p:
                ret = log(u)
                log_sims.append(ret)
        suma = s0 * exp(sum(log_sims))

        po = max((suma-strike),0) 
        payoff_final.append(po)

    call_mc = np.mean(payoff_final) * exp(-r*maturity)

    return call_mc, sims_vector, payoff_final

In [14]:
def montecarlo_diffusion(s0,strike,maturity,sigma, r, n, sims):
    """
    Calculates the price of an option call with montecarlo diffusion by introducing a set of parameters. 
    - s0: Underlying price at t0.
    - strike: Contract value of the call option.
    - maturity: period of time until the expiration of the contract in annual terms.
    - sigma: estimated constant volatility in annual terms.
    - r: estimated constant interest rate in annual terms.
    - sims: number of M simulations of the payoff of the call option.
    """
    m = r - sigma**2 / 2
    delta = maturity/n

    def create_sim(s0,r,sigma,n):
        st = s0
        def generate_value():
            nonlocal st
            for i in range(n):
                st *= exp(m * delta + sigma * sqrt(delta) * np.random.randn())
            return st
        return generate_value()

    sim_values = []
    pf_final = []
    for i in range(sims):
        st = create_sim(s0,r,sigma,n)
        sim_values.append(st)
        pf = max(st - strike, 0) 
        pf_final.append(pf)

    call_mc_2 = np.mean(pf_final) * exp(-r * maturity)
    
    return call_mc_2, sim_values, pf_final

## Inputs for functions:

In [15]:
s0 = 100
k = 100
sigma = 0.3
r = 0.045 
maturity = 1/12
n = 30
simulations = 20000
today = dt.datetime.today()

### Black & Scholes calculation for the option call price

In [18]:
test_analysis = Analysis(s0,maturity,today,maturity,r,sigma)
call_bs = test_analysis.black_scholes(s0,strike,maturity,r,sigma)[2]

In [19]:
call_bs

3.637776548563096

## Montecarlo simulatios - Option call prices:

As it can be appreciated, the call option prices gets pretty similar values for the 20.000 simulations perfomed. In case further precission is wanted, we could increase the number of simulations.

In [20]:
call_mt = montecarlo_diffusion(s0, k, maturity, sigma, r, n, simulations)
call_mt_b = montecarlo_binominal(s0, k, maturity, sigma, r,n,simulations)
print(f'Valuacion del call por Montecarlo Normal: {call_mt[0]}\nValuacion del call por Montecarlo Binomial: {call_mt_b[0]}\nValuacion del call por B&S: {call_bs}')

Valuacion del call por Montecarlo Normal: 3.6594545468702653
Valuacion del call por Montecarlo Binomial: 3.6231932857914777
Valuacion del call por B&S: 3.637776548563096


## Utilizing standard deviation as a test for the result

In [21]:
payoffs = np.array(call_mt[2]) * exp(-r * maturity)     # Retorno vector de payoffs descontados
std = np.std(payoffs) / sqrt(simulations)
payoffs2 = np.array(call_mt_b[2]) * exp(-r * maturity)
std2 = np.std(payoffs2) / sqrt(simulations)
print(f'Desvio estandar Montecarlo Normal: {round(std,4)}\nDesvio estandar Montecarlo Binomial: {round(std2,4)}')

Desvio estandar Montecarlo Normal: 0.0389
Desvio estandar Montecarlo Binomial: 0.0389


In [22]:
j_spots = np.array(call_mt[1])
m = r-(sigma**2)/2

mu1 = sum(np.log(j_spots/s0))/maturity/simulations
mu2 = sum((j_spots-s0) / s0) / maturity / simulations
mu3 = sum(np.log(j_spots/s0)**2) / maturity / simulations
mu4 = sum(((j_spots-s0) / s0)**2) / maturity / simulations

print('Testing de valores obtenidos:')
print(f'Mu 1 = {round(mu1,3)} VS {round(m,3)}\nMu 2 = {round(mu2,3)} VS {round(r,3)}\nMu 3 = {round(mu3,3)} VS {round(sigma**2,3)}\nMu 4 = {round(mu4,3)} VS {round(sigma**2,3)}')

Testing de valores obtenidos:
Mu 1 = -0.005 VS 0.0
Mu 2 = 0.041 VS 0.045
Mu 3 = 0.092 VS 0.09
Mu 4 = 0.093 VS 0.09
